## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
from __future__ import print_function
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

Using TensorFlow backend.


In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 5)                 280       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6         
Total params: 1,000,286.0
Trainable params: 1,000,286
Non-trainable params: 0.0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 18s - loss: 0.6520 - acc: 0.6363 - val_loss: 0.5889 - val_acc: 0.6931
Epoch 2/10
25000/25000 [==============================] - 17s - loss: 0.5426 - acc: 0.7259 - val_loss: 0.5326 - val_acc: 0.7280
Epoch 3/10
25000/25000 [==============================] - 17s - loss: 0.4838 - acc: 0.7683 - val_loss: 0.5012 - val_acc: 0.7502
Epoch 4/10
25000/25000 [==============================] - 16s - loss: 0.4448 - acc: 0.7943 - val_loss: 0.4809 - val_acc: 0.7610
Epoch 5/10
25000/25000 [==============================] - 16s - loss: 0.4179 - acc: 0.8092 - val_loss: 0.4677 - val_acc: 0.7725
Epoch 6/10
25000/25000 [==============================] - 17s - loss: 0.3982 - acc: 0.8212 - val_loss: 0.4572 - val_acc: 0.7788
Epoch 7/10
25000/25000 [==============================] - 17s - loss: 0.3833 - acc: 0.8283 - val_loss: 0.4534 - val_acc: 0.7832
Epoch 8/10
25000/25000 [==============================

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

24672/25000 [============================>.] - ETA: 0sTest score: 0.449169477301
Test accuracy: 0.79012


## Exercise
### Your Turn

Now do it yourself:
- Prepare the data to use sequences of length 80 rather than length 30.  Did it improve the performance?
- Try different values of the "max_features".  Can you improve the performance?
- Try smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [11]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [12]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 39s - loss: 0.5897 - acc: 0.6791 - val_loss: 0.4987 - val_acc: 0.7565
Epoch 2/10
25000/25000 [==============================] - 39s - loss: 0.4393 - acc: 0.7969 - val_loss: 0.4423 - val_acc: 0.7925
Epoch 3/10
25000/25000 [==============================] - 39s - loss: 0.3756 - acc: 0.8356 - val_loss: 0.4068 - val_acc: 0.8110
Epoch 4/10
25000/25000 [==============================] - 40s - loss: 0.3354 - acc: 0.8558 - val_loss: 0.3867 - val_acc: 0.8238
Epoch 5/10
25000/25000 [==============================] - 36s - loss: 0.3076 - acc: 0.8703 - val_loss: 0.3749 - val_acc: 0.8322
Epoch 6/10
25000/25000 [==============================] - 37s - loss: 0.2877 - acc: 0.8806 - val_loss: 0.3643 - val_acc: 0.8380
Epoch 7/10
25000/25000 [==============================] - 38s - loss: 0.2731 - acc: 0.8865 - val_loss: 0.3639 - val_acc: 0.8393
Epoch 8/10
25000/25000 [==============================

In [15]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 32s - loss: 0.6319 - acc: 0.6355 - val_loss: 0.5560 - val_acc: 0.7162
Epoch 2/10
25000/25000 [==============================] - 32s - loss: 0.5085 - acc: 0.7530 - val_loss: 0.4981 - val_acc: 0.7602
Epoch 3/10
25000/25000 [==============================] - 32s - loss: 0.4544 - acc: 0.7895 - val_loss: 0.4654 - val_acc: 0.7748
Epoch 4/10
25000/25000 [==============================] - 34s - loss: 0.4173 - acc: 0.8111 - val_loss: 0.4435 - val_acc: 0.7867
Epoch 5/10
25000/25000 [==============================] - 34s - loss: 0.3881 - acc: 0.8269 - val_loss: 0.4242 - val_acc: 0.8012
Epoch 6/10
25000/25000 [==============================] - 33s - loss: 0.3655 - acc: 0.8382 - val_loss: 0.4002 - val_acc: 0.8188
Epoch 7/10
25000/25000 [==============================] - 33s - loss: 0.3480 - acc: 0.8510 - val_loss: 0.3895 - val_acc: 0.8256
Epoch 8/10
25000/25000 [==============================

In [19]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 31s - loss: 0.3079 - acc: 0.8706 - val_loss: 0.3697 - val_acc: 0.8368
Epoch 2/10
25000/25000 [==============================] - 31s - loss: 0.3020 - acc: 0.8724 - val_loss: 0.3687 - val_acc: 0.8370
Epoch 3/10
25000/25000 [==============================] - 31s - loss: 0.2971 - acc: 0.8743 - val_loss: 0.3776 - val_acc: 0.8329
Epoch 4/10
25000/25000 [==============================] - 31s - loss: 0.2927 - acc: 0.8753 - val_loss: 0.3674 - val_acc: 0.8383
Epoch 5/10
25000/25000 [==============================] - 31s - loss: 0.2890 - acc: 0.8786 - val_loss: 0.3603 - val_acc: 0.8417
Epoch 6/10
25000/25000 [==============================] - 34s - loss: 0.2858 - acc: 0.8814 - val_loss: 0.3584 - val_acc: 0.8426
Epoch 7/10
25000/25000 [==============================] - 31s - loss: 0.2829 - acc: 0.8822 - val_loss: 0.3642 - val_acc: 0.8418
Epoch 8/10
25000/25000 [==============================